In [217]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, to_date, split, when, col, lag, lpad, sum, mean, row_number, round, count, abs, regexp_extract, date_format, lit, expr, max as spark_max, min as spark_min
from datetime import datetime, timedelta
from pyspark.sql.window import Window
from pyspark.sql import functions as F

In [218]:
spark = SparkSession.builder \
    .appName("chibao") \
    .config("spark.cores.max", "2") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

In [219]:
# Đọc dữ liệu từ bảng nguồn machungkhoan
df_lichsugia = spark.read.format("iceberg").load("stock_db.datn_lichsugia")

In [220]:
# Convert 'ngay' to DateType
df_lichsugia1= df_lichsugia.withColumn("ngay", to_timestamp(col("ngay"), "dd/MM/yyyy"))

In [221]:
# Select and rename columns to match the schema of 'fact_price_history'
df_fact_price_history = df_lichsugia1.select(
    col("symbol").alias("MaChungKhoan"),
    col("ngay").alias("NgayGiaoDich"),
    col("giamocua").alias("GiaMo").cast("bigint"),
    col("giadongcua").alias("GiaDong").cast("bigint"),
    col("giacaonhat").alias("GiaCaoNhat").cast("bigint"),
    col("giathapnhat").alias("GiaThapNhat").cast("bigint"),
    col("khoiluongkhoplenh").alias("KhoiLuong").cast("bigint")
)

In [222]:
# Chuẩn hóa lại đơn vị từ nghìn VNĐ sang VNĐ cho các cột giá
df_fact_price_history_filtered1 = df_fact_price_history.withColumn("GiaMo", col("GiaMo") * 1000) \
       .withColumn("GiaDong", col("GiaDong") * 1000) \
       .withColumn("GiaCaoNhat", col("GiaCaoNhat") * 1000) \
       .withColumn("GiaThapNhat", col("GiaThapNhat") * 1000)

In [223]:
df_fact_price_history_filtered1.show()

+------------+-------------------+-----+-------+----------+-----------+---------+
|MaChungKhoan|       NgayGiaoDich|GiaMo|GiaDong|GiaCaoNhat|GiaThapNhat|KhoiLuong|
+------------+-------------------+-----+-------+----------+-----------+---------+
|         VTZ|2024-05-31 00:00:00| 8000|   8000|      8000|       8000|   301822|
|         VTZ|2024-05-30 00:00:00| 8000|   8000|      8000|       8000|   278526|
|         VTZ|2024-05-29 00:00:00| 8000|   8000|      8000|       8000|   273500|
|         VTZ|2024-05-28 00:00:00| 8000|   8000|      8000|       8000|   265927|
|         VTZ|2024-05-27 00:00:00| 8000|   8000|      8000|       8000|   274000|
|         VTZ|2024-05-24 00:00:00| 8000|   8000|      8000|       8000|   265100|
|         VTZ|2024-05-23 00:00:00| 8000|   8000|      8000|       8000|   274000|
|         VTZ|2024-05-22 00:00:00| 8000|   8000|      8000|       8000|   269342|
|         VTZ|2024-05-21 00:00:00| 8000|   8000|      8000|       8000|   273200|
|         VTZ|20

In [224]:
# # Đọc dữ liệu từ bảng nguồn machungkhoan
# df_machungkhoan = spark.read.format("iceberg").load("stock_db.datn_machungkhoan")

In [225]:
list_ma_chung_khoan = ['AAA', 'BMC', 'BMP', 'FPT', 'NSC', 'PAC', 'BMC', 'TCT', 'TRC', 'VNM', 'VSC', 'GHA', 'HCT', 'HHC', 'NHC', 'NTP', 'GHA', 'S55', 'SAF', 'SCJ', 'VDL']   

In [226]:
df_machungkhoan = df_fact_price_history_filtered1.filter(col('MaChungKhoan').isin(list_ma_chung_khoan))

In [227]:
df = df_machungkhoan

In [228]:
# -------------------Tinh toán OBV-----------------------

In [229]:
# Khung cửa sổ để truy cập giá đóng cửa và khối lượng của phiên trước đó
window_spec = Window.partitionBy("MaChungKhoan").orderBy(col("NgayGiaoDich").asc())

# Lấy giá đóng cửa và khối lượng của phiên trước đó
df_with_prev = df.withColumn("GiaDongHomTruoc", lag("GiaDong").over(window_spec)) \
    .withColumn("KhoiLuongHomTruoc", lag("KhoiLuong").over(window_spec))

In [231]:
# Tính toán OBV
df_with_obv = df_with_prev.withColumn("OBV_change",
    when(col("GiaDongHomTruoc").isNull(), 0).  # Ngày đầu tiên
    otherwise(when(col("GiaDong") > col("GiaDongHomTruoc"), col("KhoiLuong")).
              when(col("GiaDong") < col("GiaDongHomTruoc"), -col("KhoiLuong")).
              otherwise(0))
)

In [232]:
# Tính tổng OBV
df_with_obv = df_with_obv.withColumn("OBV", sum("OBV_change").over(window_spec))

In [233]:
indicator_data_obv = df_with_obv.select(
    col("MaChungKhoan"),
    col("NgayGiaoDich"),
    col("OBV").alias("GiaTriChiBao"),
    lit("OBV").alias("LoaiChiBao")
)

In [234]:
# ----------------------------------Tính toán RSI---------------------------------

In [235]:
# Define window specifications
windowSpec_Aroonup_Aroondown = Window.partitionBy("MaChungKhoan").orderBy(col("NgayGiaoDich").cast("long")).rowsBetween(-13, 0)

In [236]:
# Tính rolling max và rolling min
df_with_rolling = df.withColumn("rolling_max", spark_max("GiaCaoNhat").over(windowSpec_Aroonup_Aroondown)) \
                    .withColumn("rolling_min", spark_min("GiaThapNhat").over(windowSpec_Aroonup_Aroondown))

In [237]:

# Tìm số ngày kể từ khi đạt giá cao nhất và thấp nhất trong 14 ngày
df_with_days_since = df_with_rolling.withColumn("row_num", row_number().over(Window.partitionBy("MaChungKhoan").orderBy(col("NgayGiaoDich").asc()))) \
                                    .withColumn("days_since_max", col("row_num") - max(when(col("GiaCaoNhat") == col("rolling_max"), col("row_num"))).over(windowSpec_Aroonup_Aroondown)) \
                                    .withColumn("days_since_min", col("row_num") - max(when(col("GiaThapNhat") == col("rolling_min"), col("row_num"))).over(windowSpec_Aroonup_Aroondown))



In [239]:
# Tính toán Aroon Up và Aroon Down
df_with_aroon = df_with_days_since.withColumn("Aroon_Up", expr("100 * (14 - days_since_max) / 14")) \
                                  .withColumn("Aroon_Down", expr("100 * (14 - days_since_min) / 14"))

In [240]:
df_with_aroon1 = df_with_aroon.filter(col("Aroon_Up").isNotNull() & col("Aroon_Down").isNotNull())

In [241]:
# Select required columns
aroon_data = df_with_aroon1.select(
    col("MaChungKhoan"),
    col("NgayGiaoDich"),
    col("Aroon_Up").alias("GiaTriChiBao"),
    lit("Aroon_Up").alias("LoaiChiBao")
).union(
    df_with_aroon.select(
        col("MaChungKhoan"),
        col("NgayGiaoDich"),
        col("Aroon_Down").alias("GiaTriChiBao"),
        lit("Aroon_Down").alias("LoaiChiBao")
    )
)


In [242]:
# ------------------Tinh toán chỉ báo A/D--------------------------

In [243]:
# Tính toán giá trị A/D cho từng ngày, xử lý trường hợp mẫu số bằng 0
df_with_ad_calculator = df.withColumn(
    "A/D",
    when(
        (col("GiaCaoNhat") - col("GiaThapNhat")) != 0,
        (((col("GiaDong") - col("GiaThapNhat")) - (col("GiaCaoNhat") - col("GiaDong"))) * col("KhoiLuong")) / (col("GiaCaoNhat") - col("GiaThapNhat"))
    ).otherwise(0)
)


In [244]:
# Định nghĩa indicatorid và chuẩn bị dữ liệu
indicator_data_ad = df_with_ad_calculator.select(
    col("MaChungKhoan"),
    col("NgayGiaoDich"),
    col("A/D").alias("GiaTriChiBao"),
    lit("A/D").alias("LoaiChiBao")  # Giả sử 2 là ID cho A/D
)

In [245]:
------------------------------------- Tính toán EMA-------------------------------------

SyntaxError: invalid syntax (2281667820.py, line 1)

In [246]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType

def calculate_ema(df, period, column='GiaDong'):
    """
    Calculate the Exponential Moving Average (EMA) for a given period.
    
    :param df: The DataFrame containing stock price data
    :param period: The period for which EMA is calculated
    :param column: The column name for which EMA is calculated
    :return: DataFrame with EMA values
    """
    alpha = 2 / (period + 1)
    
    windowSpec = Window.partitionBy("MaChungKhoan").orderBy("NgayGiaoDich")

    # Calculate initial SMA (Simple Moving Average)
    sma_window_spec = Window.partitionBy("MaChungKhoan").orderBy("NgayGiaoDich").rowsBetween(-period + 1, 0)
    df = df.withColumn('SMA', F.avg(column).over(sma_window_spec))
    
    # Initialize EMA column
    df = df.withColumn('EMA', F.lit(None).cast(DoubleType()))

    # Calculate EMA using iterative approach
    def calculate_ema_iter(closeprice, previous_ema, alpha):
        if previous_ema is None:
            return closeprice
        return (closeprice * alpha) + (previous_ema * (1 - alpha))

    # Define UDF for EMA calculation
    ema_udf = F.udf(lambda closeprice, previous_ema: calculate_ema_iter(closeprice, previous_ema, alpha), DoubleType())

    # Apply UDF to calculate EMA iteratively
    df = df.withColumn('EMA', F.coalesce(
        ema_udf(col(column), lag(col('EMA')).over(windowSpec)),
        col('SMA')
    ))
    
    return df.drop('SMA')

In [247]:
# Apply the EMA calculation
period = 20
df_with_ema = calculate_ema(df, period)

# Select required columns
ema_data_20 = df_with_ema.select(
    col("MaChungKhoan"),
    col("NgayGiaoDich"),
    col("EMA").alias("GiaTriChiBao"),
    lit("EMA" + str(period)).alias("LoaiChiBao")  # Giả sử 5 là ID cho EMA
)


In [248]:
# Apply the EMA calculation
period = 50
df_with_ema = calculate_ema(df, period)

# Select required columns
ema_data_50 = df_with_ema.select(
    col("MaChungKhoan"),
    col("NgayGiaoDich"),
    col("EMA").alias("GiaTriChiBao"),
    lit("EMA" + str(period)).alias("LoaiChiBao")  # Giả sử 5 là ID cho EMA
)


In [249]:
# Apply the EMA calculation
period = 200
df_with_ema = calculate_ema(df, period)

# Select required columns
ema_data_200 = df_with_ema.select(
    col("MaChungKhoan"),
    col("NgayGiaoDich"),
    col("EMA").alias("GiaTriChiBao"),
    lit("EMA" + str(period)).alias("LoaiChiBao")  # Giả sử 5 là ID cho EMA
)


In [251]:
# -----------------------------------Tinh toan chi bao RS-----------------------------------------

In [252]:
def calculate_rs(df, period, column='GiaDong'):
    """
    Calculate the Relative Strength (RS) for a given period.
    
    :param df: The DataFrame containing stock price data
    :param period: The period for which RS is calculated
    :param column: The column name for which RS is calculated
    :return: DataFrame with RS values
    """
    windowSpec = Window.partitionBy("MaChungKhoan").orderBy("NgayGiaoDich")

    # Calculate price change
    df = df.withColumn("price_change", col(column) - lag(col(column)).over(windowSpec))

    # Separate gains and losses
    df = df.withColumn("gain", F.when(col("price_change") > 0, col("price_change")).otherwise(0))
    df = df.withColumn("loss", F.when(col("price_change") < 0, -col("price_change")).otherwise(0))

    # Calculate average gains and losses
    avg_gain_window_spec = Window.partitionBy("MaChungKhoan").orderBy("NgayGiaoDich").rowsBetween(-period + 1, 0)
    df = df.withColumn("avg_gain", F.avg("gain").over(avg_gain_window_spec))
    df = df.withColumn("avg_loss", F.avg("loss").over(avg_gain_window_spec))

    # Calculate RS
    df = df.withColumn("RS", F.when((col("avg_loss") != 0), col("avg_gain") / col("avg_loss")).otherwise(0))

    return df

In [253]:
# Apply the RS calculation
rs_period = 14
df_with_rs = calculate_rs(df, rs_period)

# Select required columns
rs_data = df_with_rs.select(
    col("MaChungKhoan"),
    col("NgayGiaoDich"),
    col("RS").alias("GiaTriChiBao"),
    lit('RS').alias("LoaiChiBao")  # Giả sử 6 là ID cho MACD
)


In [255]:
# --------------------------Tinh toán chỉ báo RSI--------------------------------

In [256]:
def calculate_rsi(df, period):
    """
    Calculate the Relative Strength Index (RSI) for a given period.
    
    :param df: The DataFrame containing stock price data with RS values
    :param period: The period for which RSI is calculated
    :return: DataFrame with RSI values
    """
    windowSpec = Window.partitionBy("MaChungKhoan").orderBy("NgayGiaoDich")

    # Calculate RSI
    df = df.withColumn("RSI", 100 - (100 / (1 + col("RS"))))

    return df

In [257]:
# Apply the RSI calculation
rsi_period = 14
df_with_rsi = calculate_rsi(df_with_rs, rsi_period)

# Select required columns
rsi_data = df_with_rsi.select(
    col("MaChungKhoan"),
    col("NgayGiaoDich"),
    col("RSI").alias("GiaTriChiBao"),
    lit("RSI").alias("LoaiChiBao")  # Giả sử 8 là ID cho RSI
)

In [258]:
# Kết hợp tất cả các DataFrame chỉ số lại với nhau
all_indicator_data = indicator_data_obv.union(aroon_data) \
    .union(indicator_data_ad) \
    .union(ema_data_20) \
    .union(ema_data_50) \
    .union(ema_data_200) \
    .union(rs_data) \
    .union(rsi_data)

In [263]:
# Loại bỏ các bản ghi có giá trị NULL trong cột GiaTriChiBao
all_indicator_data1 = all_indicator_data.filter(col("GiaTriChiBao").isNotNull())

In [264]:
# Đổi tên cột LoaiChiBao thành TenChiBao
all_indicator_data2 = all_indicator_data1.withColumnRenamed("LoaiChiBao", "TenChiBao")


In [265]:
all_indicator_data2.show()

+------------+-------------------+------------+---------+
|MaChungKhoan|       NgayGiaoDich|GiaTriChiBao|TenChiBao|
+------------+-------------------+------------+---------+
|         AAA|2010-07-15 00:00:00|         0.0|      OBV|
|         AAA|2010-07-16 00:00:00|   -109200.0|      OBV|
|         AAA|2010-07-19 00:00:00|   -109200.0|      OBV|
|         AAA|2010-07-20 00:00:00|   -437900.0|      OBV|
|         AAA|2010-07-21 00:00:00|   -437900.0|      OBV|
|         AAA|2010-07-22 00:00:00|   -437900.0|      OBV|
|         AAA|2010-07-23 00:00:00|   -322100.0|      OBV|
|         AAA|2010-07-26 00:00:00|   -453800.0|      OBV|
|         AAA|2010-07-27 00:00:00|   -340900.0|      OBV|
|         AAA|2010-07-28 00:00:00|   -405200.0|      OBV|
|         AAA|2010-07-29 00:00:00|    103900.0|      OBV|
|         AAA|2010-07-30 00:00:00|    450200.0|      OBV|
|         AAA|2010-08-02 00:00:00|   -232600.0|      OBV|
|         AAA|2010-08-03 00:00:00|    172400.0|      OBV|
|         AAA|

In [266]:
# Ghi kết quả vào bảng `fact_stock_indicator`
all_indicator_data2.write \
    .format("jdbc") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("url", "jdbc:mysql://10.168.6.106:3306/stock") \
    .option("dbtable", "backend_factchibao") \
    .option("user", "acc_etl") \
    .option("password", "Vnpt123456") \
    .mode("append") \
    .save()